#### 15683

In [ ]:
# 1. cctv 위치 정보와 빈 칸의 개수를 저장한다.
# 2. 4^k-1(k는 cctv 수) 만큼 반복하며 각 cctv마다 감시할 수 있는 구역을 표시하고 빈칸의 수에서 차감
# 3. 2번에서 매번 사각지대의 크기를 갱신하여 최소 사각지대의 크기를 출력한다.

In [ ]:
# --------------------------------------- 테케 불러오기(메모장)
TC = []
with open("./test_cases/15683.txt", "r") as f:
    for _ in range(int(f.readline())):
        temp = []
        n, m = map(int, f.readline().split())
        temp.append((n, m))
        temp.append([list(map(int, f.readline().split())) for _ in range(n)])
        temp.append(int(f.readline()))
        TC.append(temp)
f.close()


now = TC[0]
n, m = now[0]
board = now[1]
answer = now[2]
cctv = []
empty = 0
dx = [-1, 0, 1, 0]
dy = [0, -1, 0, 1]
for i in range(n):
    temp = []
    for j in range(m):
        if board[i][j] == 0:
            empty += 1
        elif board[i][j] != 6:
            cctv.append((i, j))


output = brute_force()
print(answer == output, f"출력: {output}", f"정답: {answer}", sep="\n")
# ---------------------------------------

In [ ]:
import sys


# input = sys.stdin.readline


def brute_force():
    
    def process(x, y, k):
        nonlocal empty_case, visited
        dx = [-1, 0, 1, 0]
        dy = [0, -1, 0, 1]
        k %= 4
        while True:
            x += dx[k]
            y += dy[k]
            if not (0 <= x < n and 0 <= y < m) or board[x][y] == 6:
                return  # 사무실을 벗어나거나 벽을 만나면 종료
            elif board[x][y] != 0 or x*m+y in visited:
                continue
            visited.add(x*m+y)
            empty_case -= 1
    
    answer = empty
    for i in range(1<<(2*len(cctv))):  # 4^k개의 경우의 수를 모두 탐색 (k는 cctv 수)
        P_i = i
        empty_case = empty
        visited = set()
        for x, y in cctv:  # 케이스 i에 대하여 모든 cctv를 순회
            k = P_i % 4
            P_i //= 4
            match board[x][y]:
                case 1:
                    process(x, y, k)
                case 2:
                    process(x, y, k)
                    process(x, y, k+2)
                case 3:
                    process(x, y, k)
                    process(x, y, k+1)
                case 4:
                    process(x, y, k)
                    process(x, y, k+1)
                    process(x, y, k+2)
                case _:
                    process(x, y, k)
                    process(x, y, k+1)
                    process(x, y, k+2)
                    process(x, y, k+3)
        answer = min(answer, empty_case)
    return answer


n, m = map(int, input().split())
board = []
cctv = []
empty = 0
for i in range(n):
    temp = []
    for j, cell in enumerate(map(int, input().split())):
        if cell == 0:
            empty += 1
        elif cell != 6:
            cctv.append((i, j))
        temp.append(cell)
    board.append(temp)

print(brute_force())

True
출력: 20
정답: 20


In [ ]:
import sys


# input = sys.stdin.readline


def brute_force():
    
    def process(x, y, queue):
        nonlocal P_empty, visited
        while queue:
            k = queue.pop()
            k %= 4
            nx, ny = x, y
            while True:
                nx, ny = nx + dx[k], ny + dy[k]
                if not (0 <= nx < n and 0 <= ny < m) or board[nx][ny] == 6:
                    break  # 사무실을 벗어나거나 벽을 만나면 종료
                elif board[nx][ny] != 0 or (idx := nx*m + ny) in visited:
                    continue  # cctv이거나 이미 방문한 곳이라면 건너뜀
                visited.add(idx)
                P_empty -= 1
    
    answer = empty
    for i in range(1<<(2*len(cctv))):  # 4^k개의 경우의 수를 모두 탐색 (k는 cctv 수)
        P = i  # 임의의 경우의 수 P
        P_empty = empty
        visited = set()  # 방문 여부 기록을 위해 set 클래스 사용
        for x, y in cctv:  # 임의의 경우의 수 P에 대하여 모든 cctv를 순회
            k = P % 4  # 0 <= k <= 3
            P //= 4  # 4진법으로 해체
            match board[x][y]:  # cctv 1~5
                case 1: process(x, y, [k])
                case 2: process(x, y, [k+2, k])
                case 3: process(x, y, [k+1, k])
                case 4: process(x, y, [k+2, k+1, k])
                case _: process(x, y, [k+3, k+2, k+1, k])
        answer = min(answer, P_empty)  # 최소 사각 지대 갱신
    return answer


dx = [-1, 0, 1, 0]
dy = [0, -1, 0, 1]
n, m = map(int, input().split())
board = []
cctv = []
empty = 0
for i in range(n):
    temp = []
    for j, cell in enumerate(map(int, input().split())):
        if cell == 0:
            empty += 1
        elif cell != 6:
            cctv.append((i, j))
        temp.append(cell)
    board.append(temp)

print(brute_force())

True
출력: 20
정답: 20


#### 18808

In [ ]:
# 필요한 함수
# - 스티커 90도 회전시키는 함수
# - 노트북에 스티커 붙이는 함수(동시에 확인 작업도 진행) + 최댓값 갱신
#   - 참고사항: 새로 붙일 스티커의 가장 왼쪽 위 부분을 노트북 의 가장 왼쪽 위 부분에 부착
#   -         이미 기존의 스티커가 부착되어 있다면, 기존 스티커의 가장 오른쪽 위 부분을 기준으로 새로운 스티커를 부착
# - 브루트 포스 or 백트래킹 함수

In [ ]:
import sys


def rotate(r, c, sticker):
    return c, r, [(j, r-1-i) for i, j in sticker]


def paste(x, y, note, sticker):
    temp = set()
    for i, j in sticker:
        nx, ny = x + i, y + j
        if nx*m + ny in note:
            return False, note
        temp.add(nx*m + ny)
    return True, note | temp


def main():
    note = set()
    for _ in range(k):
        r, c = map(int, input().split())
        sticker = []
        for i in range(r):
            for j, cell in enumerate(map(int, input().split())):
                if cell == 1:
                    sticker.append((i, j))
        for _ in range(4):
            is_pasted = False
            for x in range(n-r+1):
                for y in range(m-c+1):
                    is_pasted, note = paste(x, y, note, sticker)
                    if is_pasted:
                        break
                if is_pasted:
                    break
            if is_pasted:
                break
            r, c, sticker = rotate(r, c, sticker)
    print(len(note))


if __name__ == "__main__":
    # input = sys.stdin.readline
    n, m, k = map(int, input().split())
    main()

0


#### 12100

In [ ]:
import sys


def rotate(board):
    return list(zip(*board))[::-1]


def move(k, maximum, board):
    while (k := k-1) >= -1:
        board = rotate(board)
    for i in range(n):
        row = [0] * n
        idx = 0
        for j in range(n):
            if board[i][j] != 0:
                if row[idx] == 0:
                    row[idx] = board[i][j]
                elif row[idx] == board[i][j]:
                    row[idx] *= 2
                    maximum = max(maximum, row[idx])
                    idx += 1
                else:
                    row[(idx := idx+1)] = board[i][j]
        board[i] = row
    return maximum, board


def brute_force(board, answer):
    for i in range(1024):
        P = i
        P_maximum = 0
        P_board = board  # move 함수에서 rotate 함수를 통해 새로 할당 받기 때문에 move 함수에 board를 전달할 때 copy 할 필요 없음.
        for _ in range(5):
            k = P % 4
            P //= 4
            P_maximum, P_board = move(k, P_maximum, P_board)
        answer = max(answer, P_maximum)
    print(answer)


if __name__ == "__main__":
    # input = sys.stdin.readline
    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]
    board = []
    maximum = 0
    n = int(input())
    for i in range(n):
        board.append(list(map(int, input().split())))
        maximum = max(maximum, max(board[i]))
    brute_force(board, maximum)

16
[[2, 2, 2], [4, 4, 4], [8, 8, 8]]


#### 15686

In [ ]:
import sys
from itertools import combinations


def get_distance(stores, houses):
    distance = 0
    for k, (x, y) in enumerate(stores):
        if not is_closed[k]:
            continue
        for i in range(len(houses)):
            a, b, dist = houses[i]
            distance += min(dist, abs(x-a) + abs(y-b))
            
    if distance:
        return distance
    return MAX * 2*n


def back_tracking(k, stores, houses):
    global answer
    if k == m:
        distance = get_distance(stores, [[i, j, dist] for i, j, dist in houses])
        answer = min(answer, distance)
        return
    
    for closed in range(2):
        is_closed[k] = closed
        back_tracking(k+1, stores, houses)


# input = sys.stdin.readline
n, m = map(int, input().split())

MAX = 2*n-2
houses = []
chickens = []
is_closed = [0] * m
answer = MAX * 2*n

for i in range(n):
    for j, cell in enumerate(map(int, input().split())):
        if cell == 1:
            houses.append([i, j, MAX])
        elif cell == 2:
            chickens.append((i, j))

for C in combinations(chickens, m):
    back_tracking(0, C, houses)

print(answer)

11


In [ ]:
5 1
1 2 0 0 0
1 2 0 0 0
1 2 0 0 0
1 2 0 0 0
1 2 0 0 0
11

5 2
0 2 0 1 0
1 0 1 0 0
0 0 0 0 0
2 0 0 1 1
2 2 0 1 2
10

5 3
0 0 1 0 0
0 0 2 0 1
0 1 2 0 0
0 0 1 0 0
0 0 0 0 2
5

#### 테케 불러오기

In [ ]:
# --------------------------------------- 테케 불러오기(메모장)
TC = []
with open("./test_cases/16933.txt", "r") as f:
    for _ in range(int(f.readline())):
        temp = []
        n, m, k = map(int, f.readline().split())
        temp.append((n, m, k))
        temp.append([list(map(int, f.readline().rstrip())) for _ in range(n)])
        temp.append(int(f.readline()))
        TC.append(temp)
f.close()


now = TC[6]
n, m, k = now[0]
graph = now[1]
answer = now[2]
output = bfs(n, m, k, graph)
print(answer == output, f"출력: {output}", f"정답: {answer}", sep="\n")
# ---------------------------------------